In [66]:
import kaggle as kg
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as s

In [2]:
os.environ["USERNAME"] = "aimlrl"
os.environ["KEY"] = "f9fcb3d12c2c4845724a50f0cdeab5ea"

In [3]:
kg.api.authenticate()

In [ ]:
kg.api.dataset_download_files(dataset="ehtishamsadiq/uncleaned-laptop-price-dataset",
                              path="./dataset",unzip=True)

In [38]:
data = pd.read_csv("dataset/laptopData.csv")

In [ ]:
data.head()

In [ ]:
data.isna().sum()

In [41]:
data.dropna(axis=0,inplace=True)

In [ ]:
data.isna().sum()

In [43]:
data.drop(labels=data.columns[0],axis=1,inplace=True)

In [44]:
def label_encode_column(column_name):

    data[column_name].unique()

    d = dict()
    for value in data[column_name].unique():
        d[value] = data[data[column_name] == value][data.columns[-1]].mean()

    input_target_df = pd.DataFrame(data={column_name:d.keys(),"Mean Price":d.values()})
    input_target_df.sort_values(by="Mean Price",inplace=True)
    input_target_df.reset_index(inplace=True,drop=True)

    data[column_name] = data[column_name].replace(to_replace=list(input_target_df[column_name]),
                              value=list(input_target_df.index)).infer_objects(copy=False)

In [45]:
categorical_columns = list(data.columns[0:2]) + list(data.columns[3:5]) + list(data.columns[6:9])

In [ ]:
categorical_columns

In [ ]:
for column_name in categorical_columns:

    label_encode_column(column_name)

In [ ]:
data

In [49]:
data["Ram"] = data["Ram"].apply(lambda x: float(x.split("GB")[0]))

In [ ]:
data

In [ ]:
plt.hist(data["Weight"])

In [ ]:
data["Weight"].value_counts().index[0]

In [ ]:
data["Weight"].replace(to_replace="?",value=data["Weight"].value_counts().index[0],
                       inplace=True)

In [54]:
data["Weight"] = data["Weight"].apply(lambda x: float(x.split("kg")[0]))

In [ ]:
data

In [ ]:
data["Inches"].replace(to_replace="?",value=data["Inches"].value_counts().index[0],
                       inplace=True)

In [57]:
data["Inches"] = data["Inches"].astype("float64")

In [58]:
data.drop_duplicates(inplace=True,ignore_index=True)

In [ ]:
data.shape

In [ ]:
data_corr = data.corr()
upper_traingle_mask = np.triu(np.ones_like(data_corr,dtype=bool))
sns.heatmap(data=data_corr,center=0,annot=True,square=True,xticklabels=True,yticklabels=True,mask=upper_traingle_mask)
plt.show()

In [61]:
selected_features = list(data_corr[data_corr[data.columns[-1]] > 0.5].index)

In [62]:
refined_data = data[selected_features]

In [ ]:
refined_data

In [64]:
X_transpose = np.array(refined_data.iloc[:,0:6])
y = np.array(refined_data.iloc[:,-1]).reshape(refined_data.shape[0],1)

In [ ]:
plt.xlabel("Price of a Used Laptop")
plt.ylabel("Frequency")
plt.hist(y)

In [ ]:
plt.xlabel("Price of a Used Laptop")
sns.distplot(y,fit=s.norm)

In [ ]:
plt.xlabel("Log Transformed Price of a Used Laptop")
plt.ylabel("Frequency")
plt.hist(np.log(y))

In [ ]:
plt.xlabel("Log Transformed Price of a Used Laptop")
sns.distplot(np.log(y),fit=s.norm)

In [71]:
y = np.log(y)

In [72]:
def mse(theta0,theta):

    return np.mean((y - (theta0 + np.matmul(X_transpose,theta)))**2)

In [73]:
def del_mse_by_del_theta(theta0,theta):

    error_transpose = np.transpose((theta0 + np.matmul(X_transpose,theta) - y))

    del_by_del_theta0 = np.mean(error_transpose)
    del_by_del_theta = (1/y.shape[0])*np.transpose(np.matmul(error_transpose,X_transpose))

    return [del_by_del_theta0,del_by_del_theta]

In [ ]:
epsilon = 10**(-4)
epoch_counter = 0
theta0_initial = 0
theta_initial = np.zeros((X_transpose.shape[1],1))
tol = 10**(-5)

while True:

    initial_gradients = del_mse_by_del_theta(theta0_initial,theta_initial)

    theta0_final = theta0_initial - (epsilon * initial_gradients[0])
    theta_final = theta_initial - (epsilon * initial_gradients[1])

    mse_initial_value = mse(theta0_initial,theta_initial)
    mse_final_value = mse(theta0_final,theta_final)

    if abs(mse_initial_value - mse_final_value) < tol:
        break

    epoch_counter += 1

    theta0_initial = theta0_final
    theta_initial = theta_final

    print("Epoch # {}, MSE Value = {}".format(epoch_counter,mse_initial_value))